In [ ]:
import pathlib
import shutil
import glob
import os
import re
import ftfy
import pandas as pd
import statsmodels.api as sm
import numpy as np
import sklearn.metrics as metrics

import sklearn
import sklearn.linear_model

pd.options.display.max_rows = 1000
%load_ext autoreload
%autoreload 2

In [ ]:
df = pd.read_csv("consolidated_DF_2019_2019.csv", parse_dates=["trip_start_time", "trip_stop_time"])
rng = np.random.default_rng(42)
sampled_df = df.sample(n=200000, random_state=rng)
df = sampled_df

In [ ]:
df["trip_start_daily_minutes"] = df["trip_start_time"].apply(lambda x: x.hour*60 + x.minute)
df["is_casual"] = df["is_casual"]*1

In [ ]:
df["trip_binned_daytime"] = np.floor(df["trip_start_daily_minutes"]/(1*60))
df["trip_binned_daytime"] = df["trip_binned_daytime"].astype(int)

In [ ]:
df["trip_binned_daytime"].hist(bins=100)

In [ ]:
oh_hours_df = pd.get_dummies(df["trip_binned_daytime"])
oh_station_id_df = pd.get_dummies(df["from_station_id"])

In [ ]:
df["trip_start_daily_minutes"].hist(bins=100)

In [ ]:
(df["trip_duration_seconds"]).apply(np.log).hist(bins=100)

In [ ]:
regression = sklearn.linear_model.LinearRegression()

In [ ]:
reg_features="trip_start_daily_minutes"
target = "trip_duration_seconds"

In [ ]:
hours_df = oh_hours_df.drop(columns = oh_hours_df.columns[0])

In [ ]:
station_df = oh_station_id_df.drop(columns= oh_station_id_df.columns[0])

In [ ]:
x_df = pd.concat([hours_df, station_df, df["is_casual"]], axis=1)
x_df.columns = [str(x) for x in x_df.columns]

In [ ]:
y = df[target]

In [ ]:
model = regression.fit(x_df,y)

In [ ]:
def regression_results(x, y_true, model):
    colnames = x.columns

    y_pred = model.predict(x)
    # Regression metrics
    col_coeffs = zip(model.coef_, colnames)
    col_coeffs_sorted = sorted(col_coeffs)

    r2=metrics.r2_score(y_true, y_pred)


    print(f"R^2: {round(r2,4)}")
    print(f"Intercept: {model.intercept_}")
    print(f"Top-5 coeffs: {col_coeffs_sorted[:10]}")



In [ ]:
regression_results(x_df, y, model)